In [44]:
# these seeds are both required for reproducibility
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.layers import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Input
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import keras

import os

In [45]:
#This portion was taken from zalandoresearch's utilities, to help load the data
#https://github.com/zalandoresearch/fashion-mnist/blob/master/utils/mnist_reader.py
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [46]:
#Load the data from the files
X_train, y_train = load_mnist('../input/mnist_fashion', kind='train')
X_test, y_test = load_mnist('../input/mnist_fashion', kind='t10k')
#Normalize the data to values of 0 to 1 instead of 0 to 255
X_train = X_train/255
X_test = X_test/255
#Change the y values (labels) into categorical data
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)
#Reshape the data, to work with the CNN
X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])


In [47]:
def build_model(keep_prob=0.5, optimizer='adam'):
    inputs = Input(shape=[28, 28, 1], name="input") 
    
    #Convolution 1
    conv1 = Conv2D(32, kernel_size=(3,3), activation="relu", name="conv_1")(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2), name="pool_1")(conv1)

    #Convolution 2
    conv2 = Conv2D(64, kernel_size=(3,3), activation="relu", name="conv_2")(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2), name="pool_2")(conv2)
        
    #Fully Connected Layer
    flatten = Flatten()(pool2)
    fc1 = Dense(1024, activation="relu", name="fc_1")(flatten)
    
    #output
    prediction=Dense(10, activation="softmax", name ="softmax")(fc1)

    # finalize and compile
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [63]:
def create_hyperparameters():
    batches=[10,20,30,40,50,60,70]
    optimizers = ['rmsprop','adam','adadelta']
    dropout=np.linspace(0.1,0.5,5)
    epochs = [5]
    return {"batch_size":batches, "optimizer":optimizers, "keep_prob":dropout, "epochs":epochs,}


In [74]:
model = KerasClassifier(build_fn=build_model, verbose=1)
hyperparameters=create_hyperparameters()

In [75]:
search = RandomizedSearchCV(estimator=model, param_distributions=hyperparameters, n_iter=10, n_jobs=1, cv=3, verbose=0)

In [76]:
search.fit(X_train,y_train)

Epoch 1/5
40000/40000 [==============================] - 10s 250us/step - loss: 0.1657 - acc: 0.9484
Epoch 2/5
40000/40000 [==============================] - 7s 182us/step - loss: 0.0454 - acc: 0.9852
Epoch 3/5
40000/40000 [==============================] - 7s 183us/step - loss: 0.0290 - acc: 0.9906
Epoch 4/5
40000/40000 [==============================] - 7s 183us/step - loss: 0.0202 - acc: 0.9937
Epoch 5/5
40000/40000 [==============================] - 3s 65us/step
Epoch 1/5
40000/40000 [==============================] - 10s 251us/step - loss: 0.1682 - acc: 0.9488
Epoch 2/5
40000/40000 [==============================] - 7s 181us/step - loss: 0.0449 - acc: 0.9860
Epoch 3/5
40000/40000 [==============================] - 7s 182us/step - loss: 0.0292 - acc: 0.9905
Epoch 4/5
40000/40000 [==============================] - 7s 182us/step - loss: 0.0199 - acc: 0.9937
Epoch 5/5
40000/40000 [==============================] - 3s 65us/step
Epoch 1/5
40000/40000 [==============================] - 1

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff82ffeef60>,
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'batch_size': [10, 20, 30, 40, 50, 60, 70], 'optimizer': ['rmsprop', 'adam', 'adadelta'], 'keep_prob': array([0.1, 0.2, 0.3, 0.4, 0.5]), 'epochs': [5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [77]:
print(search.best_params_)

{'optimizer': 'adadelta', 'keep_prob': 0.2, 'epochs': 5, 'batch_size': 50}


In [96]:
def build_model(keep_prob=0.1, optimizer='adadelta'):
    inputs = Input(shape=[28, 28, 1], name="input")
    
    #Convolution 1
    conv1 = Conv2D(32, kernel_size=(3,3), activation="relu", name="conv_1")(inputs)
    #batch1 = BatchNormalization(name="batch_norm_1")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name="pool_1")(conv1)

    #Convolution 2
    conv2 = Conv2D(64, kernel_size=(3,3), activation="relu", name="conv_2")(pool1)
    #batch2 = BatchNormalization(name="batch_norm_2")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), name="pool_2")(conv2)
        
    #Fully Connected Layer
    flatten = Flatten()(pool2)
    fc1 = Dense(1024, activation="relu", name="fc_1")(flatten)
    
    #output
    prediction=Dense(10, activation="softmax", name ="softmax")(fc1)

    # finalize and compile
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [123]:
#Now im going to run that model with a few more epochs because 5 is not enough, lets try 25
model=build_model()
model.fit(X_train, y_train, validation_split=0.10, batch_size=50, epochs=5, verbose=1)


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 15s 282us/step - loss: 0.1360 - acc: 0.9570 - val_loss: 0.0384 - val_acc: 0.9892
Epoch 2/5
54000/54000 [==============================] - 11s 197us/step - loss: 0.0377 - acc: 0.9884 - val_loss: 0.0336 - val_acc: 0.9905
Epoch 3/5
54000/54000 [==============================] - 11s 197us/step - loss: 0.0245 - acc: 0.9926 - val_loss: 0.0325 - val_acc: 0.9912
Epoch 4/5
54000/54000 [==============================] - 11s 197us/step - loss: 0.0170 - acc: 0.9945 - val_loss: 0.0360 - val_acc: 0.9908
Epoch 5/5
54000/54000 [==============================] - 11s 197us/step - loss: 0.0119 - acc: 0.9964 - val_loss: 0.0312 - val_acc: 0.9912


In [126]:
val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=1)

10000/10000 [==============================] - 1s 107us/step


In [127]:
print("Validation Accuracy: %f       Validation Loss: %f" % (model_accuracy, model_loss))

Validation Accuracy: 0.993200       Validation Loss: 0.042158


In [128]:
# Save your model weights
model.save("fashion_adadelta.h5")